In [164]:
import cvxopt as cvx
import numpy as np

np.set_printoptions(suppress=True)

c = cvx.matrix([-8.1, -10.8])

x1 = np.array([0.8, 0.05, 0.1, -1.0, 0.0])
x2 = np.array([0.44, 0.1, 0.36, 0.0, -1.0])
# theta1 = np.array([-1.0, 0.0, 0.0, -1.0, 1.0, 0.0, 0.0, 0.0, 0.0])
# theta2 = np.array([0.0, -1.0, 0.0, 0.0, 0.0, -1.0, 1.0, 0.0, 0.0])
b =cvx.matrix([24000.0, 2000.0, 6000.0, 0.0, 0.0])
A = np.stack([x1, x2]).T


A = cvx.matrix(A)
b = cvx.matrix(b)
c = cvx.matrix(c)

cvx.solvers.options['feastol'] = 1e-11
sol = cvx.solvers.lp(c, A, b)
print(sol['z'])


     pcost       dcost       gap    pres   dres   k/t
 0: -1.5615e+05 -4.1719e+05  3e+05  0e+00  7e-02  1e+00
 1: -2.4693e+05 -3.5373e+05  1e+05  2e-16  3e-02  6e+03
 2: -2.4913e+05 -3.2922e+05  9e+04  4e-16  2e-02  6e+03
 3: -2.7730e+05 -3.2012e+05  5e+04  5e-16  1e-02  5e+03
 4: -2.8616e+05 -3.1300e+05  3e+04  2e-16  8e-03  4e+03
 5: -2.7521e+05 -2.9460e+05  2e+04  3e-16  6e-03  3e+03
 6: -2.8636e+05 -2.8693e+05  6e+02  9e-17  2e-04  1e+02
 7: -2.8675e+05 -2.8676e+05  6e+00  5e-16  2e-06  1e+00
 8: -2.8676e+05 -2.8676e+05  6e-02  2e-16  2e-08  1e-02
 9: -2.8676e+05 -2.8676e+05  6e-04  2e-16  2e-10  1e-04
10: -2.8676e+05 -2.8676e+05  6e-06  4e-17  2e-12  1e-06
Optimal solution found.
[ 4.66e+00]
[ 8.75e+01]
[ 4.32e-10]
[ 4.18e-11]
[ 9.41e-11]



In [176]:
M = np.zeros(shape=[2+5, 2+5])
A_array = np.array(A)

z_array = np.array(sol['z'])

M[0:2, 2::] = A_array.T
M[2:2+5+1, 0:2] = np.multiply(A_array.T, z_array.T).T

N = np.zeros(shape=[2+5, 2])
N[2+0, 0] = -z_array.T[0][0]
N[2+1, 1] = -z_array.T[0][1]

M_constr_delete = []
for i in range(len(M)):
    if np.sum(np.absolute(M[i])) <= 1e-9:
        M_constr_delete.append(i)
        
        
M = np.delete(M, M_constr_delete, axis=0)
M = np.delete(M, M_constr_delete, axis=1)
N = np.delete(N, M_constr_delete, axis=0)
        

y = np.linalg.solve(M, N)
# np.sum(np.absolute(M[6]))
slope = -y

theta = np.zeros(shape=2)
# constant = np.dot(-y, -theta) + 
add_term = np.r_[sol['x'], sol['z']].T[0]

add_term_constr_delete = []
for i in range(len(add_term)):
    if np.absolute(add_term[i]) <= 1e-9:
        add_term_constr_delete.append(i)
add_term = np.delete(add_term, add_term_constr_delete)

constant = np.dot(-y, theta) + add_term


array([26206.89655135,  6896.55172405,     4.65517241,    87.51724138])